In [1]:
from glob import glob
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

In [2]:
def get_url(url):
    while True:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, "lxml")
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)
    try:
        span = soup.find_all('span')
        max_page = [s.get('title') for s in span if '页' in s.get('title', '') and '共' in s.get('title', '')][0]
        max_page = int(max_page.split()[1])
    except:
        max_page = -1
    
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    a = [a_ for a_ in a if a_.startswith('portal.php?mod=view&aid=')]
    return url, a, max_page

In [3]:
pages = []

with open('portals.json') as fopen:
    data = json.load(fopen)

for d in data:
    pages.extend(d[1])
    
pages = sorted(list(set(pages)))
len(pages)

1049

In [4]:
def parse(content):
    soup = BeautifulSoup(content, "lxml")
    
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    a = [a_ for a_ in a if a_.startswith('portal.php?mod=view&aid=')]
    return a

In [6]:
for f in tqdm(glob('pages/*.json')):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        pages.extend(parse(d))

100%|█████████████████████████████████████████| 148/148 [00:27<00:00,  5.45it/s]


In [7]:
pages = sorted(list(set(pages)))
len(pages)

62885

In [24]:
!rm -rf news
!mkdir news

In [29]:
def get_href(url):
    while True:
        try:
            r = requests.get(url, timeout = 10.0)
            break
        except:
            time.sleep(1.0)
    try:
        return r._content.decode()
    except:
        None

In [30]:
import os

max_worker = 20

for i in tqdm(range(0, len(pages), max_worker)):
    
    filename = os.path.join('news', f'{i}.json')
    if os.path.exists(filename):
        continue
    
    filtered = []
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_href, 'https://c.cari.com.my/' + url): url for url in pages[i: i + max_worker]}

        for future in as_completed(futures):
            filtered.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(filtered, fopen)

100%|███████████████████████████████████████| 3145/3145 [34:22<00:00,  1.52it/s]


In [31]:
!du -hs news

4.2G	news


In [33]:
news = glob('news/*.json')
len(news)

3145

In [35]:
with open(news[0]) as fopen_l:
    data = json.load(fopen_l)

In [38]:
json.loads(json.dumps(data[0]))

'<script src="https://vad.cari.com.my/rec/rec.php?l=c&aid=111974"></script><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<title>连外国游客都喊Awesome！今年Times Square的新年装饰很有感觉！ - 旅游资讯 - 佳礼资讯网</title>\n\n<meta property="og:site_name" content="佳礼网"/>\n<meta property="og:title" content="连外国游客都喊Awesome！今年Times Square的新年装饰很有感觉！ - 旅游资讯 -  佳礼资讯网 - "/>\n<meta property="og:description" content="今年Times Square的新年布置以…… ,佳礼资讯网" />\n<meta property="og:image" content="" />\n<meta name="twitter:image" content="">\n<meta property="fb:app_id" content="210106385698126" />\n<meta property="fb:pages" content="184572924024" />\n<meta property="fb:pages" content="325222887629144" />\n<meta property="fb:pages" content="1467392560230441" />\n<meta property="fb:pages" content="1513385025547568" />\n<meta property="fb:page

In [42]:
with open('everything.jsonl', 'w') as fopen:
    for f in news:
        try:
            with open(f) as fopen_l:
                data = json.load(fopen_l)
            for d in data:
                fopen.write(f'{json.dumps(d)}\n')
        except:
            pass